# Czyszczenie i analiza danych

Celem naszego zadania jest przygotowanie zestawu danych zawartego w pliku TitanicMess.tsv, oczyszczenie go, a następnie analiza. Zestaw zawiera informacje na temat podróżujących statkiem Titanic.


In [517]:
# Lista zaimportowanych bibliotek do przygotowania danych

import pandas as pandas
from pandas.api.types import is_numeric_dtype
import re

## Wczytanie zestawu danych

In [518]:
# wczytanie zbioru i wyświetlenie jego pierwszych rekordów

dataSet = pandas.read_csv("TitanicMess.tsv", sep="\t")
dataSet.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,"8,4583",NaN,Q,Titanic
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,"51,8625",E46,S,Titanic
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,"21,075",NaN,S,Titanic
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,"11,1333",NaN,S,Titanic
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,"30,0708",NaN,C,Titanic


## Zidentyfikowane problemy w trakcie przeglądania danych

### 1. Zbędne kolumny
Możemy zauważyć, że część kolumn jest nieistotna dla naszego zbioru danych z perspektywy analizy. Kolumna ship zawiera jedynie wartość Titanic, która jest oczywista dla zbioru danych.

In [519]:
dataSet['ship'].unique()

array(['Titanic'], dtype=object)

### 2. Powtarzające się wiersze tabeli
Możemy zauważyć powtarzający się wiersz z id = 11. Należy usunąć powielone wiersze, które zakłamują występowanie pewnego przypadku. Aby wyświetlić wszystkie powtórzenia, możemy wykorzystać funkcję `duplicated()` z biblioteki `pandas`. W poszukiwaniu powtórzonych elementów skupimy się na id, które powinno być unikalną wartością.

In [520]:
dataSet[dataSet.duplicated(['PassengerId'], keep=False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
10,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
13,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
23,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
224,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
520,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
678,225,1,1,"Hoytt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic


### 3. Błędne wartości
Następnym krokiem będzie sprawdzenie poprawności danych, które zawierają się w poszczególnych kolumnach.

#### 3.1 Wartości numeryczne

In [521]:
[is_numeric_dtype(dataSet['PassengerId']),
 is_numeric_dtype(dataSet['Survived']),
 is_numeric_dtype(dataSet['Pclass']),
 is_numeric_dtype(dataSet['Age']),
 is_numeric_dtype(dataSet['SibSp']),
 is_numeric_dtype(dataSet['Parch']),
 is_numeric_dtype(dataSet['Fare'])]

[True, True, True, False, True, True, False]

Można zauważyć, że dla wieku oraz opłat za bilet nie posiadamy jedynie wartości numerycznych. Warto zatem sprawdzić te wartości i zamienić na poprawny format. Na początek sprawdźmy wiek. Jego wartości powinny być liczbami naturalnymi.

In [522]:
for age in dataSet['Age'].dropna():
    if not age.isdigit():
        print(age)

.9
28,5
0,83
14,5
70,5
32,5
32,5
36,5
55,5
40,5
45,5
20,5
23,5
0,92
45,5
0,75
-3
-12
40,5
0,75
24,5
28,5
0,67
30,5
0,42
30,5
0,83
34,5


Okazuje się, że istnieje wiele wartości z wartościami po przecinku. Istnieją również liczby ujemne, a także przypadek, który zaczyna się od kropki. Wartości te powinny zostać zastąpione liczbami naturalnymi.

Przejdźmy zatem do wartości kolumny z cenami biletów. Wartości powinny być tam zmiennoprzecinkowymi liczbami nieujemnymi.


In [523]:
for fare in dataSet['Fare'].dropna():
    if not fare.replace(',','',1).isnumeric():
        print(fare)

-90
15,9a


Możemy zauważyć, że w kolumnie znajduje się liczba ujemna oraz wartość zawierająca literę. Należy je zamienić wartościami zmiennoprzecinkowymi nieujemnymi. Dodatkowo można też zauważyć, że pozostałe wartości tej kolumny często zawierają więcej niż dwie cyfry po przecinku, co nie jest możliwe dla wartości pieniężnych. Należałoby zaokrąglić je do dwóch miejsc po przecinku.

### 3.2 Brakujące wartości
Przy pomocy biblioteki `pandas` możemy znaleźć wartości brakujące lub określone wyrażeniem `NaN`. Posłuży nam do tego funkcja `isna()`.


In [524]:
dataSet.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          685
Embarked         2
ship             0
dtype: int64

Widzimy zatem, że kolumny `Age`, `Cabin` oraz `Embarked` zawierają brakujące wartości.

### 3.3 Wartości odstające
Wśród opisywanych kolumn jedynie kolumna z wiekiem mogłaby zawierać wartości, które byłyby zbyt duże. Wiemy już, że pojawiają się wartości ujemne, natomiast musimy usunąć również wartości zbyt duże. Przyjmijmy, że niemożliwe do osiągnięcia jest posiadanie więcej niż 115 lat.

In [525]:
# Należy dodać lstrip(), ponieważ jeden wynik zaczyna się od kropki
for age in dataSet['Age'].dropna():
    if float(age.lstrip('.').replace(',','.',1)) > 115:
        print(age)

4435
250


Uzyskaliśmy zatem dwa wyniki, które nie są dla człowieka osiągalne w kwestii wieku. Należy zastanowić się, w jaki sposób trzeba je zamienić.

### 3.4 Znaki niepożądane

Już pierwsze wiersze całego zbioru danych pokazują, że wśród danych osobowych pasażerów mogą znajdować się znaki niealfanumeryczne.

In [526]:
for name in dataSet['Name']:
    x = re.match(r'.*[^A-Za-z\s,.\"\'\-()/].*', name)
    if x:
        print(name)


Sandstrom, Miss. Marguerite Ru&5$$
Sandstrom, Miss. Marguerite Ru&5$$
Sandstrom, Miss. Marguerite Ru&5$$


Udało nam się pokazać, że tylko jedno imię zawiera nieodpowiednie znaki.

### 3.5 Literówki
W swoich rozważaniach skupmy się na elementach, które nie zawierają zbyt wielu unikalnych wartości. Jeżeli jest ich zbyt dużo, trudno byłoby znaleźć w każdym możliwe zamiany liter.

In [527]:
dataSet.nunique() 

PassengerId    888
Survived         2
Pclass           3
Name           889
Sex              6
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           250
Cabin          145
Embarked         6
ship             1
dtype: int64

Spośród powyższych wartości możemy odrzucić `Survived` oraz `Pclass`. Pierwsza kolumna zawiera jedynie jedynki oraz a druga jedną z trzech dostępnych klas. Widzimy zatem, że nie pojawiają się tam żadne nieoczekiwane wartości. Warto przyjrzeć się na pewno kolumnie `Sex`, która prawdopodobnie zawiera zbyt wiele opcji. Sytuacja ma się podobnie z trzema innymi kolumnami. Warto zatem przejrzeć możliwe wartości:

In [528]:
dataSet['Sex'].unique()

array(['male', 'female', 'malef', 'mal', 'fem', 'femmale'], dtype=object)

Możemy zauważyć, że w kolumnie `Sex` pojawiają się literówki. Naszym zadaniem będzie zatem podmiana błędnych wyrazów.

In [529]:
dataSet['SibSp'].unique()


array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

Powyższa kolumna zawiera liczbę rodzeństwa/małżonków, którzy znajdują się również na pokładzie statku. Nie widać tutaj nieprawidłowych wartości.

In [530]:
dataSet['Parch'].unique()

array([0, 1, 2, 5, 3, 4], dtype=int64)

Kolejna kolumna zawiera informacje o liczbie rodziców/dzieci danego pasażera znajdujących się na pokładzie. Również nie widać tutaj niestandardowych wartości.

In [531]:
dataSet['Embarked'].unique()

array(['S', 'C', 'Q', 'So', nan, 'Co', 'Qe'], dtype=object)

Z informacji o zbiorze danych dowiadujemy się, że są jedynie 3 dostępne porty (C = Cherbourg, Q = Queenstown, S = Southampton). Część z wartości zawiera jednak po jednej dodatkowej literze, co jedynie wprowadza zamieszanie i należy je ujednolicić. Występują również braki danych w dwóch wierszach. Zaobserwowaliśmy to już poprzednich analizach.